In [2]:
import os
import pandas as pd 
import glob

cwd = os.getcwd()
results_dir = f"{cwd}/results"

In [3]:
letters = ["a"]#,"b","c","d","e"]
model_names = ["microsoft/deberta-v3-base","FacebookAI/xlm-roberta-base"]#,"dslim/bert-base-NER-uncased"]# ["answerdotai/ModernBERT-base"]
best_params_dct = {l:{mn:[] for mn in model_names} for l in letters}
for l in letters:
    for mn in model_names:
        df = pd.read_csv(f"{cwd}/results/{l}/sghead/hyptuning_{mn.split('/')[-1]}.csv")
        el_vals = df["values_0"]
        f1_vals = df["values_1"]
        id_besteval = list(el_vals).index(el_vals.min())
        id_bestf1 = list(f1_vals).index(f1_vals.max())
        if id_besteval == id_bestf1:
            best_params_dct[l][mn].append({
                "eval_loss": df.loc[id_besteval, "values_0"],
                "overall_f1": df.loc[id_besteval, "values_1"],
                "batch_size": df.loc[id_besteval, "params_batch_size"],
                "lr": df.loc[id_besteval, "params_lr"],
                "max_length": df.loc[id_besteval, "params_max_length"],
                "weight_decay": df.loc[id_besteval, "params_weight_decay"]
            })
        else:
            for idx in [id_besteval, id_bestf1]:
                best_params_dct[l][mn].append({
                    "eval_loss": df.loc[idx, "values_0"],
                    "overall_f1": df.loc[idx, "values_1"],
                    "batch_size": df.loc[idx, "params_batch_size"],
                    "lr": df.loc[idx, "params_lr"],
                    "max_length": df.loc[idx, "params_max_length"],
                    "weight_decay": df.loc[idx, "params_weight_decay"]
                })
best_params_dct

{'a': {'microsoft/deberta-v3-base': [{'eval_loss': np.float64(0.2889979836695334),
    'overall_f1': np.float64(0.6319853688274741),
    'batch_size': np.int64(8),
    'lr': np.float64(0.000777261011589),
    'max_length': np.int64(512),
    'weight_decay': np.float64(0.0196272722522528)}],
  'FacebookAI/xlm-roberta-base': [{'eval_loss': np.float64(0.316255635023117),
    'overall_f1': np.float64(0.6196139598362255),
    'batch_size': np.int64(32),
    'lr': np.float64(0.0006583230277609),
    'max_length': np.int64(512),
    'weight_decay': np.float64(0.0796458140678382)}]}}

In [12]:
el_vals = df["values_0"]
f1_vals = df["values_1"]

print(el_vals.min(), f1_vals.max())
print(list(el_vals).index(el_vals.min()), list(f1_vals).index(f1_vals.max()))
df.iloc[5]

0.3422350794076919 0.6062322946175638
5 5


Unnamed: 0                                                        5
number                                                            5
values_0                                                   0.342235
values_1                                                   0.606232
datetime_start                           2026-02-19 23:59:19.612194
datetime_complete                        2026-02-20 00:02:22.288162
duration                                     0 days 00:03:02.675968
params_batch_size                                                 8
params_lr                                                    0.0002
params_max_length                                               256
params_weight_decay                                         0.07455
system_attrs_NSGAIISampler:generation                             0
state                                                      COMPLETE
Name: 5, dtype: object

In [15]:
df.iloc[[5, 6, 9, 11, 16, 21, 24, 26]]

,Unnamed: 0,number,values_0,values_1,datetime_start,datetime_complete,duration,params_batch_size,params_lr,params_max_length,params_weight_decay,system_attrs_NSGAIISampler:generation,state
5,5,5,0.342235,0.606232,2026-02-19 23:59:19.612194,2026-02-20 00:02:22.288162,0 days 00:03:02.675968,8,0.000200,256,0.074550,0,COMPLETE
6,6,6,0.349024,0.565610,2026-02-20 00:02:22.289119,2026-02-20 00:07:42.885841,0 days 00:05:20.596722,8,0.000066,512,0.013451,0,COMPLETE
9,9,9,0.366925,0.525376,2026-02-20 00:19:19.510936,2026-02-20 00:25:23.063015,0 days 00:06:03.552079,32,0.000185,512,0.089817,0,COMPLETE
11,11,11,0.380471,0.522280,2026-02-20 00:31:15.419041,2026-02-20 00:36:12.225143,0 days 00:04:56.806102,8,0.000050,256,0.079142,0,COMPLETE
16,16,16,0.368861,0.544962,2026-02-20 00:59:05.603885,2026-02-20 01:02:08.525661,0 days 00:03:02.921776,8,0.000106,512,0.098567,0,COMPLETE
21,21,21,0.369268,0.534701,2026-02-20 01:25:26.669672,2026-02-20 01:31:30.828811,0 days 00:06:04.159139,32,0.000275,512,0.075243,0,COMPLETE
24,24,24,0.358257,0.577707,2026-02-20 01:43:41.796942,2026-02-20 01:47:30.457506,0 days 00:03:48.660564,8,0.000101,512,0.097176,0,COMPLETE
26,26,26,0.362504,0.569178,2026-02-20 01:53:34.044074,2026-02-20 01:58:41.943548,0 days 00:05:07.899474,8,0.000095,512,0.072571,0,COMPLETE


In [3]:
l = "e"
htype = "sghead"
mn = "microsoft/deberta-v3-base"
df = pd.read_csv(f"{cwd}/results/{l}/{htype}/hyptuning_{mn.split('/')[-1]}.csv")
el_vals = df["values_0"]
f1_vals = df["values_1"]
print(el_vals.min(), f1_vals.max())
id_besteval = list(el_vals).index(el_vals.min())
id_bestf1 = list(f1_vals).index(f1_vals.max())
print(id_besteval,id_bestf1)

df
#df.iloc[[3,4,15,25,39,48]]
#df.iloc[[19,21,27,32,48]]
#df.iloc[[11,21,47]]

2.0023475955514347 0.0043080317007296
39 9


,Unnamed: 0,number,values_0,values_1,datetime_start,datetime_complete,duration,params_batch_size,params_lr,params_max_length,params_weight_decay,system_attrs_NSGAIISampler:generation,state
0,0,0,2.764962,0.000000,2026-02-22 12:42:25.774189,2026-02-22 12:49:33.341301,0 days 00:07:07.567112,16,0.000008,256,0.004705,0,COMPLETE
1,1,1,2.052437,0.000000,2026-02-22 12:49:33.343040,2026-02-22 12:52:40.008450,0 days 00:03:06.665410,32,0.000225,256,0.036730,0,COMPLETE
2,2,2,2.134349,0.000000,2026-02-22 12:52:40.010203,2026-02-22 12:59:50.759728,0 days 00:07:10.749525,32,0.000034,512,0.086630,0,COMPLETE
3,3,3,2.046856,0.000000,2026-02-22 12:59:50.761405,2026-02-22 13:02:02.469800,0 days 00:02:11.708395,16,0.000162,256,0.021977,0,COMPLETE
4,4,4,2.049078,0.000000,2026-02-22 13:02:02.471218,2026-02-22 13:06:08.447998,0 days 00:04:05.976780,16,0.000060,512,0.051599,0,COMPLETE
5,5,5,2.052246,0.000000,2026-02-22 13:06:08.449683,2026-02-22 13:08:20.823415,0 days 00:02:12.373732,32,0.000423,512,0.088115,0,COMPLETE
6,6,6,7.114062,0.000000,2026-02-22 13:08:20.825070,2026-02-22 13:15:18.561825,0 days 00:06:57.736755,8,0.000002,512,0.093628,0,COMPLETE
7,7,7,2.048080,0.000000,2026-02-22 13:15:18.563537,2026-02-22 13:18:10.763280,0 days 00:02:52.199743,32,0.000259,256,0.063203,0,COMPLETE
8,8,8,39.712234,0.003649,2026-02-22 13:18:10.765047,2026-02-22 13:25:34.161549,0 days 00:07:23.396502,32,0.000001,512,0.023147,0,COMPLETE
9,9,9,31.871797,0.004308,2026-02-22 13:25:34.163351,2026-02-22 13:32:55.559509,0 days 00:07:21.396158,32,0.000002,256,0.085685,0,COMPLETE


In [ ]:
l = "e"
htype = "sghead"
mn = "FacebookAI/xlm-roberta-base"
df = pd.read_csv(f"{cwd}/results/{l}/{htype}/hyptuning_{mn.split('/')[-1]}.csv")
el_vals = df["values_0"]
f1_vals = df["values_1"]
print(el_vals.min(), f1_vals.max())
id_besteval = list(el_vals).index(el_vals.min())
id_bestf1 = list(f1_vals).index(f1_vals.max())
print(id_besteval,id_bestf1)

df
#df.iloc[[0,9,49]]
#df.iloc[[2,12,32,40,45,48]]
#df.iloc[[2,14,22,24,27,31,43]]
#df.iloc[[1,10,21]]

0.8661486945654217 0.3944896390000865
10 10


,Unnamed: 0,number,values_0,values_1,datetime_start,datetime_complete,duration,params_batch_size,params_lr,params_max_length,params_weight_decay,system_attrs_NSGAIISampler:generation,state
1,1,1,0.911103,0.345417,2026-02-22 17:58:29.769222,2026-02-22 18:04:21.393884,0 days 00:05:51.624662,8,0.000234,512,0.097342,0,COMPLETE
10,10,10,0.866149,0.394490,2026-02-22 18:58:13.791915,2026-02-22 19:02:09.319098,0 days 00:03:55.527183,8,0.000680,512,0.041033,0,COMPLETE
21,21,21,0.978036,0.308233,2026-02-22 20:09:52.049320,2026-02-22 20:16:29.479529,0 days 00:06:37.430209,16,0.000334,512,0.081864,0,COMPLETE


In [10]:
l = "e"
htype = "sghead"
mn = "answerdotai/ModernBERT-base"
df = pd.read_csv(f"{cwd}/results/{l}/{htype}/hyptuning_{mn.split('/')[-1]}.csv")
el_vals = df["values_0"]
f1_vals = df["values_1"]
print(el_vals.min(), f1_vals.max())
id_besteval = list(el_vals).index(el_vals.min())
id_bestf1 = list(f1_vals).index(f1_vals.max())
print(id_besteval,id_bestf1)

df
#df.iloc[[14,21,24,32,37,44,49]]
#df.iloc[[2,3,17,22,24]]
#df.iloc[[0,5,16,19,30,32]]
#df.iloc[[21,7]]
df.iloc[[7]]

0.6826528569062551 0.3835253338770517
7 7


,Unnamed: 0,number,values_0,values_1,datetime_start,datetime_complete,duration,params_batch_size,params_lr,params_max_length,params_weight_decay,system_attrs_NSGAIISampler:generation,state
7,7,7,0.682653,0.383525,2026-02-24 07:59:05.666326,2026-02-24 08:02:13.333376,0 days 00:03:07.667050,8,0.000711,768,0.077844,0,COMPLETE


In [28]:
###
l = "b"
htype = "sghead"
mn = "microsoft/deberta-v3-base"
df = pd.read_csv(f"{cwd}/results/{l}/{htype}/hyptuning_{mn.split('/')[-1]}.csv")

df[df["params_lr"]<0.00001]

,Unnamed: 0,number,values_0,values_1,datetime_start,datetime_complete,duration,params_batch_size,params_lr,params_max_length,params_weight_decay,system_attrs_NSGAIISampler:generation,state
1,1,1,4.971346,0.000000,2026-02-21 15:22:09.568731,2026-02-21 15:28:52.365064,0 days 00:06:42.796333,32,0.000007,256,0.071341,0,COMPLETE
2,2,2,35.274770,0.001625,2026-02-21 15:28:52.366312,2026-02-21 15:35:44.576526,0 days 00:06:52.210214,32,0.000001,256,0.053602,0,COMPLETE
5,5,5,14.478009,0.000045,2026-02-21 15:44:09.882680,2026-02-21 15:50:55.720352,0 days 00:06:45.837672,32,0.000003,512,0.078659,0,COMPLETE
7,7,7,35.882469,0.002547,2026-02-21 15:53:21.934063,2026-02-21 16:00:17.179184,0 days 00:06:55.245121,32,0.000001,256,0.007413,0,COMPLETE
10,10,10,16.176523,0.000544,2026-02-21 16:05:19.239574,2026-02-21 16:12:03.875534,0 days 00:06:44.635960,32,0.000004,512,0.060756,0,COMPLETE
13,13,13,29.066944,0.001248,2026-02-21 16:19:23.456406,2026-02-21 16:26:16.428466,0 days 00:06:52.972060,32,0.000002,512,0.085155,0,COMPLETE
14,14,14,29.129720,0.002263,2026-02-21 16:26:16.429840,2026-02-21 16:33:09.666478,0 days 00:06:53.236638,32,0.000002,512,0.062635,0,COMPLETE
21,21,21,4.926430,0.000000,2026-02-21 17:00:54.226357,2026-02-21 17:07:22.326636,0 days 00:06:28.100279,8,0.000002,256,0.044659,0,COMPLETE
24,24,24,14.667240,0.000617,2026-02-21 17:20:22.271619,2026-02-21 17:27:06.146046,0 days 00:06:43.874427,16,0.000002,256,0.043766,0,COMPLETE
27,27,27,3.275201,0.000000,2026-02-21 17:40:05.524473,2026-02-21 17:46:45.226634,0 days 00:06:39.702161,16,0.000005,512,0.061690,0,COMPLETE
